# Tarefa 01 Módulo 23 Combinação de Modelos


## 1. Monte um passo a passo para o Bagging:

### Processo de Bagging: Um Passo a Passo

#### 1° Bootstrap

Primeiro, vamos entender o Bootstrap. Imagine que temos um dataset com um certo número de linhas e colunas. Aplicando o Bootstrap, geramos vários novos datasets a partir do original. Cada novo dataset tem o mesmo número de linhas e colunas, mas pode incluir linhas repetidas e possivelmente excluir algumas linhas do dataset original. Continuamos este processo até gerar o número necessário de datasets para nosso problema.

#### 2° Base Learners

A próxima etapa é a criação dos "Base Learners". Aqui, treinamos diferentes modelos de aprendizado, como árvores de decisão, regressões logísticas, regressões lineares, etc., para cada um dos datasets gerados pelo Bootstrap. Dessa forma, obtemos um modelo para cada dataset.

#### 3° Bagging (Aggregating)

Por fim, chegamos à etapa de agregação, também conhecida como Bagging. Nesta fase, combinamos os resultados dos modelos. Se estivermos lidando com um problema de classificação binária, pegamos o valor mais frequente (1 ou 0) entre os modelos. Se o problema for de regressão (não binário), agregamos os resultados calculando a média dos valores preditos pelos modelos.



# 2). Monte um passo a passo para o Bagging:

Este é um processo que ajuda a melhorar a robustez e a precisão de um modelo final, combinando os resultados de vários modelos treinados em diferentes amostras do dataset original.

# 3). Implemente em python o código do bagging:
    - Bootstrap
    - Modelagem
    - Agregação
    

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
import matplotlib.pyplot as plt

In [2]:
data = load_iris()
X, y = data.data, data.target
n_estimators = 10  # Número de modelos base (base learners)
data

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

## Bootstrap

In [3]:
def bootstrap_samples(X, y, n_samples):
    samples = []
    for i in range(n_samples):
        X_resampled, y_resampled = resample(X, y, random_state=i+1) 
        samples.append((X_resampled, y_resampled))
    return samples

## Modelagem:

In [4]:
def train_base_learners(samples):
    base_learners = []
    for i, (X_sample, y_sample) in enumerate(samples):
        model = DecisionTreeClassifier(random_state=i+1)
        model.fit(X_sample, y_sample)
        base_learners.append(model)
        # Visualizar parte das amostras no Bootstrap
        if i < 3:
            print(f"Amostra {i+1} - Tamanho: {len(X_sample)}")
            print(f"X_sample:\n{X_sample[:5]}\n")
            print(f"y_sample:\n{y_sample[:5]}\n")
    return base_learners

## Agregação:

In [5]:
def aggregate_predictions(base_learners, X_test):
    predictions = np.zeros((len(X_test), len(base_learners)))
    for i, model in enumerate(base_learners):
        predictions[:, i] = model.predict(X_test)
    
    # Para classificação, usar votação por maioria
    y_pred = np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=1, arr=predictions)
    
    # Visualizar resultado da votação
    print(f"Resultado da Votação:\n{y_pred}\n")
    
    return y_pred


## Processo de Bagging:

In [6]:
def bagging(X, y, n_estimators, X_test):
    # Bootstrap
    samples = bootstrap_samples(X, y, n_estimators)
    
    # Modelagem
    base_learners = train_base_learners(samples)
    
    # Agregação
    y_pred = aggregate_predictions(base_learners, X_test)
    
    return y_pred

## Dividir os Dados em Treino e Teste:

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Aplicar Bagging e Avaliar Performance:

In [8]:
y_pred = bagging(X_train, y_train, n_estimators, X_test)

# Avaliar a performance
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Amostra 1 - Tamanho: 120
X_sample:
[[7.7 3.8 6.7 2.2]
 [6.3 2.9 5.6 1.8]
 [6.7 3.1 4.7 1.5]
 [4.4 3.  1.3 0.2]
 [5.2 4.1 1.5 0.1]]

y_sample:
[2 2 1 0 0]

Amostra 2 - Tamanho: 120
X_sample:
[[5.7 2.5 5.  2. ]
 [5.5 2.4 3.7 1. ]
 [6.5 3.  5.5 1.8]
 [4.4 3.  1.3 0.2]
 [5.6 3.  4.5 1.5]]

y_sample:
[2 1 2 0 1]

Amostra 3 - Tamanho: 120
X_sample:
[[7.4 2.8 6.1 1.9]
 [7.7 2.8 6.7 2. ]
 [4.8 3.4 1.6 0.2]
 [5.8 2.8 5.1 2.4]
 [4.4 3.  1.3 0.2]]

y_sample:
[2 2 0 2 0]

Resultado da Votação:
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]

Accuracy: 100.00%
